In [ ]:
!apt update
!apt install -y chromium-browser chromium-chromedriver
!pip install selenium undetected-chromedriver pandas


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,855 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,

In [ ]:
import os
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd


cereri_links = [
    "https://singur-in-instanta.ro/model-cerere-de-apel",
    "https://singur-in-instanta.ro/model-cerere-de-chemare-in-judecata",
    "https://singur-in-instanta.ro/model-cerere-reconventionala",
    "https://singur-in-instanta.ro/raspuns-la-intampinare",
    "https://singur-in-instanta.ro/model-plangere-impotriva-procesului-verbal-de-constatare-a-contraventiei",
    "https://singur-in-instanta.ro/model-cerere-eliberare-certificat-de-grefa",
    "https://singur-in-instanta.ro/model-cerere-anulare-casatorie",
    "https://singur-in-instanta.ro/model-cerere-modificare-regim-matrimonial",
    "https://singur-in-instanta.ro/model-cerere-autorizare-casatorie-minor-ce-a-implinit-varsta-de-16-ani",
    "https://singur-in-instanta.ro/model-cerere-despagubiri-pentru-cheltuielile-facute-contractate-in-vederea-casatoriei-precum-si-pentru-prejudiciile-cauzate-ca-urmare-a-ruperii-logodnei-in-mod-abuziv",
    "https://singur-in-instanta.ro/model-cerere-restituire-bunuri-oferite-ca-daruri-in-considerarea-logodnei",
    "https://singur-in-instanta.ro/model-cerere-de-legalizare-a-unei-hotarari-judecatoresti"
    "https://singur-in-instanta.ro/model-cerere-partaj-succesoral-iesire-din-indiviziune-in-materie-de-mostenire"
    "https://singur-in-instanta.ro/cerere-privind-dispunerea-de-catre-instanta-a-partajului-bunurilor-comune"
    "https://singur-in-instanta.ro/model-cerere-de-divort-intemeiata-pe-o-separare-in-fapt-care-a-durat-cel-putin-2-ani",
    "https://singur-in-instanta.ro/model-cerere-de-divort-intemeiata-pe-motive-de-sanatate",
    "https://singur-in-instanta.ro/model-cerere-de-divort-prin-acordul-sotilor",
    "https://singur-in-instanta.ro/model-cerere-reconventionala",
    "https://singur-in-instanta.ro/model-intampinare-la-cererea-de-chemare-in-judecata",
    "https://singur-in-instanta.ro/model-cerere-de-chemare-in-judecata",
    "https://singur-in-instanta.ro/model-cerere-de-acordare-de-facilitati-sub-forma-reduceriiesalonarii-sisau-amanarii-pentru-plata-taxelor-judicare-de-timbru-de-catre-persoanele-juridice",
    "https://singur-in-instanta.ro/cerere-de-eliberare-a-unui-nou-certificat-de-stare-civila",
    "https://singur-in-instanta.ro/cerere-de-divort-la-primarie",
    "https://singur-in-instanta.ro/cerere-de-restituire-a-taxei-judiciare-de-timbru",
    "https://singur-in-instanta.ro/cerere-de-chemare-in-judecata-pentru-stabilirea-pensiei-de-intretinere-datorata-copilului-major-aflat-in-continuarea-studiilor",
    "https://singur-in-instanta.ro/cerere-de-chemare-in-judecata-pentru-stabilirea-pensiei-de-intretinere-datorata-copilului-minor",
    "https://singur-in-instanta.ro/cerere-de-chemare-in-judecata-pentru-stabilirea-pensiei-de-intretinere-datorata-copilului-minor",
    "https://singur-in-instanta.ro/Model-Declaratie-ajutor-public-judiciar",
    "https://singur-in-instanta.ro/Model-Cerere-de-reexaminare-admitere-in-parte-respingere-ajutor-public-judiciar",
    "https://singur-in-instanta.ro/Model-Cerere-ajutor-public-sub-forma-scutirii-reducerii-esalonarii-amanarii-de-la-plata-taxelor-judiciare",
    "https://singur-in-instanta.ro/Model-Cerere-ajutor-public-plata-onorariu-avocat-expert-traducator-interpret-executor"
]

OUTPUT_DIR = "data/cereri_finale"
FILES_DIR = os.path.join(OUTPUT_DIR, "files")
os.makedirs(FILES_DIR, exist_ok=True)

def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "_", name.strip())

data = []

for url in cereri_links:
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")


        title_tag = soup.find("h1") or soup.find("h2") or soup.find("title")
        title = title_tag.get_text(strip=True) if title_tag else "N/A"


        content_tag = soup.find("div", class_="entry-content") or soup.find("article") or soup.find("body")
        content = content_tag.get_text("\n", strip=True) if content_tag else ""


        filename = sanitize_filename(title) + ".txt"
        file_path = os.path.join(FILES_DIR, filename)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(content)

        data.append({
            "title": title,
            "page_url": url,
            "file_path": file_path,
            "content": content
        })

        print(f"Salvat cerere: {title}")

    except Exception as e:
        print(f"Eroare la {url}: {e}")

df = pd.DataFrame(data)
csv_path = os.path.join(OUTPUT_DIR, "dataset_cereri_finale.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f"\nSalvat {len(df)} cereri în CSV: {csv_path}")


Salvat cerere: Model cerere de apel
Salvat cerere: Model cerere de chemare în judecată
Salvat cerere: Model Cerere reconvențională
Salvat cerere: Răspuns la întâmpinare
Salvat cerere: Model plângere împotriva procesului-verbal de constatare a contravenției
Salvat cerere: Model cerere eliberare certificat de grefă plângere contravențională
Salvat cerere: Model cerere anulare căsătorie
Salvat cerere: Model cerere modificare regim matrimonial
Salvat cerere: Model cerere autorizare căsătorie minor ce a împlinit vârsta de 16 ani
Salvat cerere: Model cerere despăgubiri pentru cheltuielile făcute/ contractate în vederea căsătoriei, precum și pentru prejudiciile cauzate ca urmare a ruperii logodnei în mod abuziv
Salvat cerere: Model cerere restituire bunuri oferite ca daruri în considerarea logodnei
Eroare la https://singur-in-instanta.ro/model-cerere-de-legalizare-a-unei-hotarari-judecatorestihttps://singur-in-instanta.ro/model-cerere-partaj-succesoral-iesire-din-indiviziune-in-materie-de-mos

In [ ]:
!pip install pdfminer.six


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.1 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import base64
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from pathlib import Path
from pdfminer.high_level import extract_text

# === CONFIG ===
BASE = "https://legeaz.net"
HEADERS = {"User-Agent": "Mozilla/5.0"}

ROOT_DIR = Path("data/cerere")
PDF_DIR = ROOT_DIR / "pdfs"
TXT_DIR = ROOT_DIR / "txts"
OUTPUT_DIR = Path("data/cereri_finale")
CSV_PATH = OUTPUT_DIR / "dataset_cereri_finale.csv"

# Creează directoarele
PDF_DIR.mkdir(parents=True, exist_ok=True)
TXT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


# === UTILS ===
def sanitize_filename(name):
    return re.sub(r"[^A-Za-z0-9_-]", "_", name.strip()) or "file"


def get_article_links():
    """Get all article links from all pages of legeaz.net/modele that contain 'cerere'."""
    links = set()
    pages = [f"{BASE}/modele"] + [f"{BASE}/modele/pagina-{i}" for i in range(2, 19)]
    print(f"🔎 Caut doar cereri pe {len(pages)} pagini...")

    for page in pages:
        try:
            r = requests.get(page, headers=HEADERS, timeout=10)
            soup = BeautifulSoup(r.text, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if href.startswith("/modele/") and "cerere" in href.lower():
                    links.add(urljoin(BASE, href))
            print(f"✅ {page} → {len(links)} linkuri de cereri total")
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ Eroare la {page}: {e}")

    print(f"🔗 Total cereri găsite: {len(links)}")
    return list(links)


def find_pdf_intermediate(article_url):
    """Find intermediate PDF download page (contains Base64 fragment)."""
    r = requests.get(article_url, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")
    a = soup.select_one('a[href*="/descarca-pdf#"]')
    if a:
        return urljoin(BASE, a["href"])
    return None


def decode_pdf_url(intermediate_url):
    """Decode Base64 fragment to get real PDF URL."""
    fragment = urlparse(intermediate_url).fragment
    if not fragment:
        return None
    try:
        decoded_bytes = base64.b64decode(fragment)
        decoded_url = decoded_bytes.decode("utf-8")
        return urljoin(BASE, decoded_url)
    except Exception:
        return None


def download_pdf(pdf_url, out_path):
    if out_path.exists():
        return out_path
    r = requests.get(pdf_url, headers=HEADERS)
    if r.status_code == 200 and r.content.startswith(b"%PDF"):
        out_path.write_bytes(r.content)
        print("⬇️  PDF descărcat:", out_path.name)
        return out_path
    else:
        print("⚠️ PDF invalid sau eșuat:", pdf_url)
        return None


def pdf_to_txt(pdf_path, txt_path):
    try:
        text = extract_text(str(pdf_path))
        txt_path.write_text(text.strip(), encoding="utf-8")
        print("📝 Convertit:", pdf_path.name, "→", txt_path.name)
    except Exception as e:
        print("⚠️ Conversie eșuată:", e)


def extract_html_text(article_url, txt_path):
    """Fallback dacă nu există PDF."""
    r = requests.get(article_url, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")
    text = soup.get_text(separator="\n", strip=True)
    txt_path.write_text(text.strip(), encoding="utf-8")
    print("💾 Salvat text HTML:", txt_path.name)


# === MAIN ===
def main():
    article_links = get_article_links()

    # Încarcă CSV existent, dacă există
    if CSV_PATH.exists():
        df_existing = pd.read_csv(CSV_PATH)
        existing_urls = set(df_existing["page_url"].tolist())
    else:
        df_existing = pd.DataFrame()
        existing_urls = set()

    data_new = []

    for link in article_links:
        if link in existing_urls:
            continue  # deja în CSV

        name = sanitize_filename(link)
        pdf_path = PDF_DIR / f"{name}.pdf"
        txt_path = TXT_DIR / f"{name}.txt"

        intermediate_pdf = find_pdf_intermediate(link)
        if intermediate_pdf:
            pdf_url = decode_pdf_url(intermediate_pdf)
            if pdf_url:
                pdf_file = download_pdf(pdf_url, pdf_path)
                if pdf_file:
                    pdf_to_txt(pdf_file, txt_path)
        else:
            extract_html_text(link, txt_path)

        try:
            content = txt_path.read_text(encoding="utf-8", errors="ignore")
        except:
            content = ""
        data_new.append({
            "title": name,
            "page_url": link,
            "file_path": str(txt_path),
            "content": content
        })

        print(f"✅ Cerere salvată: {name}")
        time.sleep(1)

    # Actualizează CSV
    if data_new:
        df_new = pd.DataFrame(data_new)
        df_final = pd.concat([df_existing, df_new], ignore_index=True)
        df_final.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
        print(f"\n✅ Adăugate {len(df_new)} cereri noi în CSV: {CSV_PATH}")
    else:
        print("\nℹ️ Nicio cerere nouă de adăugat (toate existau deja).")


if __name__ == "__main__":
    main()


🔎 Caut doar cereri pe 18 pagini...
✅ https://legeaz.net/modele → 2 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-2 → 17 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-3 → 48 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-4 → 76 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-5 → 90 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-6 → 99 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-7 → 116 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-8 → 129 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-9 → 141 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-10 → 153 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-11 → 154 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-12 → 160 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-13 → 165 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-14 → 169 linkuri de cereri total
✅ https://legeaz.net/modele/pagina-15 

In [ ]:
def categorize_ramura(title: str) -> str:
    """Încadrează automat o cerere într-o ramură de drept, pe baza cuvintelor-cheie din titlu."""
    t = title.lower()


    if any(x in t for x in [
        "divort", "căsătorie", "casatorie", "logodn", "pensie",
        "minor", "intretinere", "custodie", "adop", "familie",
        "regim matrimonial", "separare in fapt", "autoritate parinteasca"
    ]):
        return "Dreptul familiei"


    elif any(x in t for x in [
        "succesor", "moștenire", "mostenire", "partaj", "indiviziune",
        "iesire din indiviziune", "masa succesorala", "mostenitor"
    ]):
        return "Drept succesoral / Patrimonial"


    elif any(x in t for x in [
        "contravent", "amenda", "proces verbal", "plangere contraventionala"
    ]):
        return "Drept contravențional"


    elif any(x in t for x in [
        "apel", "recurs", "reexaminare", "ajutor public", "taxe judiciare",
        "legalizare", "intampinare", "cerere reconventionala", "chemare in judecata",
        "ordonanta presedintiala", "contestatie in anulare"
    ]):
        return "Drept procesual civil"


    elif any(x in t for x in [
        "certificat", "stare civila", "eliberare", "restituire", "autorizatie",
        "plangere prealabila", "anulare act administrativ", "petitie",
        "reconstituire", "functie publica", "primarie", "prefectura", "directie"
    ]):
        return "Drept administrativ / Necontencios"


    elif any(x in t for x in [
        "munca", "salariat", "angajator", "salariu", "demisie",
        "concediere", "reintegrare", "contract individual de munca",
        "recalculare vechime", "disciplinara", "spor"
    ]):
        return "Dreptul muncii"


    elif any(x in t for x in [
        "societate", "asociat", "administrator", "dizolvare", "lichidare",
        "firma", "pfa", "contract comercial", "inregistrare societate"
    ]):
        return "Drept comercial / Societar"


    elif any(x in t for x in [
        "impozit", "taxa", "anaf", "finante", "contestatie fiscala",
        "obligatii fiscale", "declaratie fiscala"
    ]):
        return "Drept fiscal"


    elif any(x in t for x in [
        "executare silita", "poprire", "somatie", "plangere executor",
        "contestatie la executare", "executori"
    ]):
        return "Executare silită"


    elif any(x in t for x in [
        "contract", "restituire", "obligatie", "pretentii", "despagubiri",
        "daune", "creanta", "plata", "reparatie", "reziliere", "anulare"
    ]):
        return "Drept civil general"


    elif any(x in t for x in [
        "consumator", "garantie", "produse", "servicii", "publicitate inselatoare"
    ]):
        return "Dreptul protecției consumatorului"


    elif any(x in t for x in [
        "plangere penala", "infractiune", "furt", "inselaciune", "denunt",
        "urmărire penală", "parchet", "plângere împotriva procurorului"
    ]):
        return "Drept penal"


    else:
        return "Alte ramuri de drept / Neclasificat"


In [ ]:
def categorize_tip_actiune(title):
    t = title.lower()


    if any(x in t for x in [
        "chemare in judecata", "cerere de chemare", "actiune principala",
        "introducere actiune", "cerere introductiva", "cerere principala"
    ]):
        return "Cerere inițială"


    elif any(x in t for x in [
        "apel", "recurs", "reexaminare", "contestatie in anulare", "revizuire"
    ]):
        return "Cale de atac"


    elif any(x in t for x in [
        "reconventionala", "cerere reconventionala"
    ]):
        return "Cerere reconvențională"


    elif any(x in t for x in [
        "intampinare", "raspuns la intampinare", "pozitie procesuala",
        "aparare", "opozitie"
    ]):
        return "Întâmpinare / Răspuns"


    elif any(x in t for x in [
        "ajutor public", "certificat", "legalizare", "eliberare", "adeverinta",
        "autorizare", "restituire", "reexaminare taxa", "inregistrare",
        "declaratie", "notificare", "petitie", "plangere prealabila"
    ]):
        return "Cerere necontencioasă / Administrativă"


    elif any(x in t for x in [
        "plangere contraventionala", "proces verbal de contraventie"
    ]):
        return "Plângere contravențională"


    elif any(x in t for x in [
        "plangere penala", "sesizare", "denunt", "cerere de interventie"
    ]):
        return "Plângere penală / Sesizare"


    elif any(x in t for x in [
        "contestatie la executare", "executare silita", "poprire",
        "plangere executor", "suspendare executare", "anulare executare"
    ]):
        return "Cerere în executare silită"


    elif any(x in t for x in [
        "partaj", "lichidare", "iesire din indiviziune", "impartire"
    ]):
        return "Cerere de partaj / Împărțire"


    elif any(x in t for x in [
        "constatare", "constatatorie", "recunoastere"
    ]):
        return "Cerere de constatare"


    elif any(x in t for x in [
        "suspendare", "amanare", "prelungire", "revocare"
    ]):
        return "Cerere procedurală (suspendare / amânare / revocare)"


    elif any(x in t for x in [
        "taxa judiciara", "scutire", "reducere", "esalonare", "amanare plata"
    ]):
        return "Cerere privind taxele judiciare"


    elif any(x in t for x in [
        "pensie", "drepturi salariale", "ajutor de somaj", "indemnizatie",
        "recalculare pensie", "plata salariu"
    ]):
        return "Cerere pentru drepturi sociale / salariale"


    elif any(x in t for x in [
        "divort", "separare in fapt", "custodie", "minor", "regim matrimonial"
    ]):
        return "Cerere de familie (divorț / custodie)"


    elif any(x in t for x in [
        "pretentii", "despagubiri", "daune", "obligatie de plata", "restituire suma"
    ]):
        return "Cerere de despăgubiri / pretenții"


    elif any(x in t for x in [
        "interventie", "chemare in garantie", "introducere tert"
    ]):
        return "Cerere de intervenție / garanție"


    elif "ordonanta presedintiala" in t:
        return "Ordonanță președințială"


    else:
        return "Altele"


In [ ]:
import pandas as pd

# Încarcă tot CSV-ul existent în DataFrame
CSV_PATH = "data/cereri_finale/dataset_cereri_finale.csv"
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

# Verifică primele rânduri
print(df.head())

# Verifică coloanele disponibile
print(df.columns)


                                               title  \
0                               Model cerere de apel   
1                Model cerere de chemare în judecată   
2                       Model Cerere reconvențională   
3                             Răspuns la întâmpinare   
4  Model plângere împotriva procesului-verbal de ...   

                                            page_url  \
0  https://singur-in-instanta.ro/model-cerere-de-...   
1  https://singur-in-instanta.ro/model-cerere-de-...   
2  https://singur-in-instanta.ro/model-cerere-rec...   
3  https://singur-in-instanta.ro/raspuns-la-intam...   
4  https://singur-in-instanta.ro/model-plangere-i...   

                                           file_path  \
0  data/cereri_finale/files/Model cerere de apel.txt   
1  data/cereri_finale/files/Model cerere de chema...   
2  data/cereri_finale/files/Model Cerere reconven...   
3  data/cereri_finale/files/Răspuns la întâmpinar...   
4  data/cereri_finale/files/Model plângere împ

In [ ]:
df["ramura_dreptului"] = df["title"].apply(categorize_ramura)
df["tip_actiune"] = df["title"].apply(categorize_tip_actiune)

# Salvează CSV-ul actualizat
csv_path = os.path.join(OUTPUT_DIR, "dataset_cereri_finale_categorizat.csv")
df.to_csv(csv_path, index=False, encoding="utf-8-sig")

print(f" Fișier categorizat salvat: {csv_path}")
print(df[["title", "ramura_dreptului", "tip_actiune"]])

 Fișier categorizat salvat: data/cereri_finale/dataset_cereri_finale_categorizat.csv
                                                 title  \
0                                 Model cerere de apel   
1                  Model cerere de chemare în judecată   
2                         Model Cerere reconvențională   
3                               Răspuns la întâmpinare   
4    Model plângere împotriva procesului-verbal de ...   
..                                                 ...   
230  https___legeaz_net_modele_incheiere-incuviinta...   
231  https___legeaz_net_modele_cerere-contestatie-e...   
232  https___legeaz_net_modele_incheiere-solutionar...   
233  https___legeaz_net_modele_cerere-incuviintare-...   
234  https___legeaz_net_modele_cerere-recalculare-p...   

                        ramura_dreptului  \
0                  Drept procesual civil   
1    Alte ramuri de drept / Neclasificat   
2    Alte ramuri de drept / Neclasificat   
3    Alte ramuri de drept / Neclasificat  

In [ ]:
import shutil
from google.colab import files

# Path to the folder you want to download
folder_path = "data"

# Create a zip archive
shutil.make_archive("cereri_finale", 'zip', folder_path)

# Download the zip file
files.download("cereri_finale.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>